<a href="https://colab.research.google.com/github/Mahnytskyy/I.Magnitsky-FIT-2-8-LR-Python/blob/main/%D0%9B%D0%B0%D0%B17_%D0%9C%D0%B0%D0%B3%D0%BD%D0%B8%D1%86%D1%8C%D0%BA%D0%B8%D0%B9_%D0%86%D0%B2%D0%B0%D0%BD_%D0%A4%D0%86%D0%A24_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Лаб7. Магницький Іван ФІТ4-8
# Завдання 1
!pip install scikit-surprise

In [14]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')

# Виведіть перші 5 рядків завантаженого датасету.
for row in data.raw_ratings[:5]:
    print(row)

('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [15]:
# Реалізуйте два алгоритми для рекомендаційної системи на основі цього датасету.
# Можна вибрати будь-які алгоритми з бібліотеки Surprise.

In [16]:
trainset, testset = train_test_split(data, test_size=0.2)

param_grid_svd = {'n_factors': [50, 100],
                  'lr_all': [0.005, 0.010],
                  'reg_all': [0.02, 0.05]}

param_grid_knn = {'k': [20, 40],
                  'min_k': [1, 5],
                  'sim_options': {'name': ['cosine', 'pearson'],
                                  'user_based': [False, True]}}

In [17]:
# Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.
grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

In [18]:
# найкращий алгоритм на основі середньої абсолютної помилки (MAE)
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Найкращий MAE для SVD: 0.7240753924356355
Найкращий MAE для KNNBasic: 0.8031609563367145
Вибраний алгоритм: SVD


In [19]:
#рекомендації (10 фільмів) для конкретного користувача
best_model.fit(trainset)

user_id = str(196)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 196: 32
Топ-10 фільмів, рекомендованих для користувача:
Фільм 318 з прогнозованим рейтингом 4.52
Фільм 64 з прогнозованим рейтингом 4.50
Фільм 169 з прогнозованим рейтингом 4.50
Фільм 408 з прогнозованим рейтингом 4.40
Фільм 657 з прогнозованим рейтингом 4.39
Фільм 12 з прогнозованим рейтингом 4.38
Фільм 483 з прогнозованим рейтингом 4.35
Фільм 1194 з прогнозованим рейтингом 4.35
Фільм 515 з прогнозованим рейтингом 4.34
Фільм 963 з прогнозованим рейтингом 4.34
